# SCORE Backend API Server
基于Omnizart的音频处理后端服务

## 步骤:
1. 安装依赖环境 (micromamba + omnizart)
2. 安装Python库 (FastAPI, uvicorn等)
3. 上传main.py代码
4. 启动API服务器
5. 获取ngrok公共URL并配置前端

## 1. 安装Micromamba和Omnizart环境

In [ ]:
%%bash
# 安装micromamba
if [ ! -f "${HOME}/micromamba/bin/micromamba" ]; then
    echo "Installing micromamba..."
    curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj -C /usr/local bin/micromamba
else
    echo "Micromamba already installed"
fi

# 初始化micromamba
/usr/local/bin/micromamba shell init -s bash -p ~/micromamba
source ~/.bashrc

# 创建omni环境（如果不存在）
if ! /usr/local/bin/micromamba env list | grep -q "omni"; then
    echo "Creating omni environment..."
    /usr/local/bin/micromamba create -n omni python=3.8 -c conda-forge -y
fi

echo "Environment setup complete!"

In [ ]:
# 在omni环境中安装Omnizart和相关库
!/usr/local/bin/micromamba run -n omni pip install omnizart
!/usr/local/bin/micromamba run -n omni pip install pretty-midi music21 librosa essentia
!/usr/local/bin/micromamba run -n omni pip install demucs spleeter

## 2. 安装Web服务器依赖

In [ ]:
# 安装FastAPI和相关库
!/usr/local/bin/micromamba run -n omni pip install fastapi uvicorn[standard] python-multipart
!/usr/local/bin/micromamba run -n omni pip install pyngrok nest-asyncio
!/usr/local/bin/micromamba run -n omni pip install ffmpeg-python

## 3. 创建工作目录和上传代码

In [ ]:
import os
from pathlib import Path

# 创建工作目录
work_dir = Path("/content/score_backend")
work_dir.mkdir(exist_ok=True)
os.chdir(work_dir)

# 创建必要的子目录
(work_dir / "uploads").mkdir(exist_ok=True)
(work_dir / "outputs").mkdir(exist_ok=True)

print(f"Working directory: {work_dir}")
print(f"Please upload your main.py file to this directory")

In [ ]:
# 方式1: 直接在此处粘贴main.py代码
# 将上面提供的main.py内容保存为文件

main_py_code = '''
# 在这里粘贴完整的main.py代码
'''

with open('main.py', 'w', encoding='utf-8') as f:
    f.write(main_py_code)

print("main.py created successfully!")

In [ ]:
# 方式2: 从Google Drive挂载
from google.colab import drive
drive.mount('/content/drive')

# 如果你已经在Drive中有main.py，可以复制过来
# !cp /content/drive/MyDrive/score/main.py /content/score_backend/

## 4. 配置ngrok认证（可选，提高稳定性）

In [ ]:
# 如果你有ngrok账号，可以设置authtoken以获得更稳定的连接
# 注册账号: https://dashboard.ngrok.com/signup
# 获取token: https://dashboard.ngrok.com/get-started/your-authtoken

from pyngrok import ngrok

# 替换为你的ngrok authtoken
# ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN_HERE")

print("Ngrok configured (optional)")

## 5. 启动SCORE API服务器 🚀

In [ ]:
# 启动服务器
# 运行后会输出ngrok公共URL
# 将该URL复制到前端项目的.env文件中作为VITE_API_URL

!/usr/local/bin/micromamba run -n omni python main.py

## 6. 测试API
服务器启动后，你可以在新的cell中测试API

In [ ]:
import requests

# 替换为你的ngrok URL
API_URL = "https://xxxx-xx-xxx-xxx-xx.ngrok-free.app"

# 测试健康检查
response = requests.get(f"{API_URL}/api/health")
print("Health check:", response.json())

# 测试文件上传（需要准备一个测试音频文件）
# with open('test_audio.mp3', 'rb') as f:
#     files = {'file': f}
#     response = requests.post(f"{API_URL}/api/upload", files=files)
#     print("Upload response:", response.json())

## 使用说明

### 前端配置
1. 复制ngrok输出的公共URL
2. 在前端项目根目录创建`.env`文件
3. 添加: `VITE_API_URL=https://xxxx-xx-xxx-xxx-xx.ngrok-free.app`
4. 重启前端开发服务器

### API端点
- `GET /api/health` - 健康检查
- `POST /api/upload` - 上传文件
- `GET /api/status/{job_id}` - 查询处理状态
- `GET /api/result/{job_id}` - 获取处理结果
- `GET /api/download/{job_id}/{file_type}` - 下载文件
- `POST /api/cancel/{job_id}` - 取消任务

### 注意事项
- Colab会话有时间限制（12小时）
- ngrok免费版URL会在每次重启时改变
- 上传大文件时注意Colab的存储限制
- 生产环境建议使用专用服务器